In [1]:
from datetime import datetime
import secondaryindex
from client import Client
import exceptions
import requests
import sqlite3
import pandas as pd
import time

# API Key & Secret number
my_binance = Client('05WV0wxZV0OdWuSPurpFmH3fiSk1XEukIF6em0HXAHmE4qlCVWdvuM5PeghYExzw',
                    'J3xvURK7z5APzMaDB6jjelRvmh9oJ6jnZXIdqntoUkhfxu6YK8r15hKEnUjM48iF')

#################################################################################################
#                                       낚시 준비물 챙기기                                        #
#################################################################################################
### Part1. 지속적인 알고리즘을 수행하기 전, pre-processing 작업

# 접속시간, 잔고 출력 추가하기
print ("\n접 속 시 간     :   ",datetime.now().replace(microsecond=0))

all_currencies_count = len(my_binance.get_products()['data']) # 전체 상장 갯수
print ("\nBinance에 거래가능한 종목 : ",all_currencies_count)

# step1. Get list of currencies
currencies_btc_market = [] # The list of BTC market currencies
currencies_eth_market = [] # The list of ETH market currencies
currencies_bnb_market = [] # The list of BNB market currencies
currencies_usdt_market = [] # The list of USDT market currencies
get_products_data = my_binance.get_products()['data']

# 최대한 call function 을 줄일 수 있도록 하자.(Server 에 접근을 많이 할수록 속도는 느려짐)
for i in range(0,all_currencies_count):
    if get_products_data[i]['symbol'][-3:] == 'BTC':
        currencies_btc_market.append(get_products_data[i]['symbol'])
        #print (get_products_data[i]['symbol'][-3:],"1")
    elif get_products_data[i]['symbol'][-3:] == 'ETH':
        currencies_eth_market.append(get_products_data[i]['symbol'])
        #print (get_products_data[i]['symbol'][-3:],"2")
    elif get_products_data[i]['symbol'][-3:] == 'BNB':
        currencies_bnb_market.append(get_products_data[i]['symbol'])
        #print (get_products_data[i]['symbol'][-3:],"3")
    elif get_products_data[i]['symbol'][-3:] == 'SDT':
        currencies_usdt_market.append(get_products_data[i]['symbol'])
        #print (get_products_data[i]['symbol'][-3:],"4")
    else:
        print ("Cant't classify")

print ("BTC  시장 상장 종목 갯수  : ",len(currencies_btc_market))
print ("ETH  시장 상장 종목 갯수  : ",len(currencies_eth_market))
print ("BNB  시장 상장 종목 갯수  : ",len(currencies_bnb_market))
print ("USDT 시장 상장 종목 갯수  : ",len(currencies_usdt_market))

# step2. Declare the variables often used
currencies_btc_market_count = len(currencies_btc_market)
currencies_eth_market_count = len(currencies_eth_market)
currencies_bnb_market_count = len(currencies_bnb_market)
currencies_usdt_market_count = len(currencies_usdt_market)

# outlier percent 지정
outlier_percent = -0.0

# the rate of profit 지정
rate_of_profit = 1.02

print ("\n이탈률이 ", "{0:.3}".format(outlier_percent*100),"% 로 지정되었습니다.")
print ("수익률이 ", "{0:.3}".format((rate_of_profit-1)*100),"% 로 지정되었습니다.")


default_historical_start_str = '1400000000000'




접 속 시 간     :    2018-04-28 02:53:50

Binance에 거래가능한 종목 :  305
BTC  시장 상장 종목 갯수  :  122
ETH  시장 상장 종목 갯수  :  120
BNB  시장 상장 종목 갯수  :  55
USDT 시장 상장 종목 갯수  :  8

이탈률이  -0.0 % 로 지정되었습니다.
수익률이  2.0 % 로 지정되었습니다.


In [6]:
# 5분 데이터 저장
print ("\n현재로부터 모든 종목의 과거 데이터(5분봉)를 로드하고 있습니다. 약 1시간의 시간을 소요합니다.")
datalist_past_min5 = {'_time': [], '_symbol': [], '_open': [], '_high': [], '_low': [], '_close': [],
                      '_base_buy': [], '_base_sell': [], '_quote_buy': [], '_quote_sell': []}
min5_last_data = {}
for i in range(0, currencies_btc_market_count):
    try:
        tmp_min5_data = my_binance.get_historical_klines(symbol=currencies_btc_market[i],
                                                         interval=my_binance.KLINE_INTERVAL_5MINUTE,
                                                         start_str=default_historical_start_str)
        
        for j in range(0, len(tmp_min5_data)):
            # datalist_past_min5 생성 및 데이터베이스 저장
            datalist_past_min5['_time'].append(tmp_min5_data[j][0]/1000)
            datalist_past_min5['_symbol'].append(currencies_btc_market[i])
            datalist_past_min5['_open'].append(tmp_min5_data[j][1])
            datalist_past_min5['_high'].append(tmp_min5_data[j][2])
            datalist_past_min5['_low'].append(tmp_min5_data[j][3])
            datalist_past_min5['_close'].append(tmp_min5_data[j][4])
            datalist_past_min5['_base_buy'].append(tmp_min5_data[j][9])
            datalist_past_min5['_base_sell'].append(round(float(tmp_min5_data[j][5])-float(tmp_min5_data[j][9]),2))
            datalist_past_min5['_quote_buy'].append(tmp_min5_data[j][10])
            datalist_past_min5['_quote_sell'].append(round(float(tmp_min5_data[j][7])-float(tmp_min5_data[j][10]),2))
                
            df = pd.DataFrame(datalist_past_min5, columns=['_time', '_symbol', '_open', '_high',
                                                           '_low', '_close', '_base_buy', '_base_sell',
                                                           '_quote_buy','_quote_sell'], index=datalist_past_min5['_time'])

            con = sqlite3.connect("C:\\Users\\dolid\\database_binance\\stock_min5.db")
            df.to_sql(currencies_btc_market[i], con, if_exists='append')
            con.close()
            
        print (currencies_btc_market[i]," 종목 데이터 저장을 완료했습니다.")
    
    except IndexError as e:
        print(e)
        print("min5_last_data error")

print ("현재로부터 모든 종목의 과거 데이터(5분봉)를 로드를 완료하였습니다.")



현재로부터 모든 종목의 과거 데이터를 로드하고 있습니다. 약 5분의 시간을 소요합니다.


KeyboardInterrupt: 

In [6]:
# 1시간 데이터 저장
print ("\n현재로부터 모든 종목의 과거 데이터(1시간봉)를 로드하고 있습니다. 약 1시간의 시간을 소요합니다.")
print (datetime.now())
for i in range(0, currencies_btc_market_count):
    try:
        tmp_hour1_data = my_binance.get_historical_klines(symbol=currencies_btc_market[i],
                                                         interval=my_binance.KLINE_INTERVAL_1HOUR,
                                                         start_str=default_historical_start_str)
        datalist_past_hour1 = {'_time': [], '_symbol': [], '_open': [], '_high': [], '_low': [], '_close': [],
                      '_base_buy': [], '_base_sell': [], '_quote_buy': [], '_quote_sell': []}
        
        for j in range(0, len(tmp_hour1_data)):
            # datalist_past_hour1 생성 및 데이터베이스 저장
            datalist_past_hour1['_time'].append(tmp_hour1_data[j][0]/1000)
            datalist_past_hour1['_symbol'].append(currencies_btc_market[i])
            datalist_past_hour1['_open'].append(tmp_hour1_data[j][1])
            datalist_past_hour1['_high'].append(tmp_hour1_data[j][2])
            datalist_past_hour1['_low'].append(tmp_hour1_data[j][3])
            datalist_past_hour1['_close'].append(tmp_hour1_data[j][4])
            datalist_past_hour1['_base_buy'].append(tmp_hour1_data[j][9])
            datalist_past_hour1['_base_sell'].append(round(float(tmp_hour1_data[j][5])-float(tmp_hour1_data[j][9]),2))
            datalist_past_hour1['_quote_buy'].append(tmp_hour1_data[j][10])
            datalist_past_hour1['_quote_sell'].append(round(float(tmp_hour1_data[j][7])-float(tmp_hour1_data[j][10]),2))
                
            df = pd.DataFrame(datalist_past_hour1, columns=['_time', '_symbol', '_open', '_high',
                                                           '_low', '_close', '_base_buy', '_base_sell',
                                                           '_quote_buy','_quote_sell'], index=datalist_past_hour1['_time'])

            con = sqlite3.connect("C:\\Users\\dolid\\database_binance\\stock_hour1.db")
            df.to_sql(currencies_btc_market[i], con, if_exists='replace')
            con.close()
            
        print (currencies_btc_market[i]," 종목 데이터 저장을 완료했습니다.")
    
    except IndexError as e:
        print(e)
        print("hour1_last_data error")

print ("현재로부터 모든 종목의 과거 데이터(1시간봉)를 로드를 완료하였습니다.")
print (datetime.now())


현재로부터 모든 종목의 과거 데이터(1시간봉)를 로드하고 있습니다. 약 1시간의 시간을 소요합니다.
2018-04-28 03:48:16.685996
BNBBTC  종목 데이터 저장을 완료했습니다.
NULSBTC  종목 데이터 저장을 완료했습니다.


KeyboardInterrupt: 

In [4]:
tmp_hour1_data = my_binance.get_klines(symbol=currencies_btc_market[i],
                                              interval=my_binance.KLINE_INTERVAL_1HOUR,
                                              limit=limit_get_klines_hour1)

In [5]:
tmp_hour1_data

[[1524564000000,
  '0.00001038',
  '0.00001046',
  '0.00001028',
  '0.00001040',
  '6169524.00000000',
  1524567599999,
  '64.05548857',
  2192,
  '2802191.00000000',
  '29.10935891',
  '0'],
 [1524567600000,
  '0.00001039',
  '0.00001040',
  '0.00001022',
  '0.00001028',
  '7864920.00000000',
  1524571199999,
  '80.90317738',
  2921,
  '3857087.00000000',
  '39.70468271',
  '0'],
 [1524571200000,
  '0.00001028',
  '0.00001057',
  '0.00001028',
  '0.00001045',
  '8816077.00000000',
  1524574799999,
  '92.10341109',
  3593,
  '4789387.00000000',
  '50.03092376',
  '0'],
 [1524574800000,
  '0.00001045',
  '0.00001093',
  '0.00001042',
  '0.00001043',
  '19655911.00000000',
  1524578399999,
  '209.69426684',
  5557,
  '11176126.00000000',
  '119.39621893',
  '0'],
 [1524578400000,
  '0.00001044',
  '0.00001058',
  '0.00001027',
  '0.00001047',
  '7537904.00000000',
  1524581999999,
  '78.43371938',
  2585,
  '3127694.00000000',
  '32.59112107',
  '0'],
 [1524582000000,
  '0.00001049',
  '

In [25]:
res = my_binance.get_historical_klines(symbol='LTCBTC',
                                interval = my_binance.KLINE_INTERVAL_1HOUR,
                                start_str = '1420000000000',
                                end_str = '1524564000000')

In [29]:
print (datetime.fromtimestamp(res[0][0]/1000).strftime("%Y-%m-%d %H:%M:%S"))

2017-07-14 13:00:00


In [27]:
res[0][0]/1000a

1500004800.0

In [30]:
for i in range(0,currencies_btc_market_count):
    tmp_hour1_data = my_binance.get_historical_klines(symbol=currencies_btc_market[i],
                                                      interval=my_binance.KLINE_INTERVAL_1HOUR,
                                                      start_str='1400000000000')
    print (currencies_btc_market_count[i],"시작일 : ",datetime.fromtimestamp(tmp_hour1_data[0][0]/1000).strftime("%Y-%m-%d %H:%M:%S"))

KeyboardInterrupt: 

In [31]:
tmp_hour1_data = my_binance.get_historical_klines(symbol=currencies_btc_market[i],
                                                      interval=my_binance.KLINE_INTERVAL_1HOUR,
                                                      start_str='1400000000000')

In [33]:
len(tmp_hour1_data)

6862

In [4]:
con.close()